In [1]:
import numpy as np
import pandas as pd

In [20]:
import csv

data = []
with open('data.csv', 'r') as file:
    my_reader = csv.reader(file, delimiter=',')
    for row in my_reader:
        data.append(row)

In [104]:
data[0]

['id',
 'diagnosis',
 'radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'radius_se',
 'texture_se',
 'perimeter_se',
 'area_se',
 'smoothness_se',
 'compactness_se',
 'concavity_se',
 'concave points_se',
 'symmetry_se',
 'fractal_dimension_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst',
 'fractal_dimension_worst',
 '']

In [22]:
data_ = data[1:]

In [23]:
data_ = [x[1:] for x in data_]

In [36]:
# return X_train, X_test, y_train, y_test

def train_test_split(x, y, test_size=0.3):
    return (x[:int(len(x)*(1 - test_size))], x[int(len(x) * (1-test_size)):],
            y[:int(len(y)*(1-test_size))], y[int(len(y)*(1-test_size)):])

In [37]:
x_train, x_test, y_train, y_test = train_test_split(X, y)

In [41]:
f'x train size: {len(x_train)} , x test size {len(x_test)} '

'x train size: 398 , x test size 171 '

In [75]:
def preprocess_y(y):
    
    for i in range(len(y)):
        if y[i] == 'B':
            y[i] = 1
        elif y[i] == 'M':
            y[i] = 0
    
    return y

In [76]:
y_train = preprocess_y(y_train)

In [92]:
def to_int(x):
    for i in range(len(x)):
        for j in range(len(x[i])):
            x[i][j] = float(x[i][j])
            
    return x

In [93]:
x_train = to_int(x_train)

In [46]:
def are_positive(target_attr):
    for i in range(len(target_attr)):
        if i == 0:
            return False
    return True

def are_negative(target_attr):
    for i in range(len(target_attr)):
        if i == 1:
            return False
    return True

def most_common_value(target_attr):
    cntr0 = 0
    cntr1 = 1
    for i in range(len(target_attr)):
        if i == 1:
            cntr1 += 1
        if i == 0:
            cntr0 += 1
    return 1 if cntr1 > cntr0 else 0

def subset_satisfied(examples, attr_index, value, splitter=None):
    res = []
    
    if splitter == 'l':
        for field in examples:
            if field[attr_index] < value:
                res.append(field)
        return res
    
    if splitter == 'g':
        for field in examples:
            if field[attr_index] >= value:
                res.append(field)
                
        return res
    
    else:
        for field in examples:
            if field[attr_index] == value:
                res.append(field)
            
    return res
        

In [62]:
def get_positive_samples(x, y):
    res = []
    
#     assert len(x) == len(y)
    
    for i in range(len(x)):
        if y[i] == 1:
            res.append(x[i])
    return res

def get_negative_samples(x, y):
    res = []
    for i in range(len(x)):
        if y[i] == 0:
            res.append(x[i])
    return res

def get_all_values(x, col_index):
    return list(set(x[i][col_index] for i in range(len(x))))

In [100]:
import math

def log2(x):
    assert x != 0
    return math.log(x) / math.log(2)

def entropy(x, y):
    pos_samples = get_positive_samples(x, y)
    print('pos samples', pos_samples)
    pos_len = len(pos_samples)
    total_len = len(x)
    temp = pos_len / total_len
    
    print("temp ", temp)
    
    return - temp * log2(temp) - (1-temp) * log2(1 - temp)

def gain(x, y, ind):
    
    unique_vals = get_all_values(x, ind)
    res = 0
    
    for i in range(len(unique_vals)):
        Ai = subset_satisfied(x, ind, unique_vals[i])
        print("Ai ", Ai)
        res += entropy(Ai, y) * len(Ai) / len(x)
        
    return entropy(x, y) - res

    
def get_gains(x, y, n_attrs=32):
    gains = []
    
    for i in range(1, n_attrs):
        gains.append(gain(x, y, i))
        
    return gains

In [97]:
# class Node():
    
#     def __init__(self, data):
#         self.data = data
#         self.children = []
#     def append_node(self, node):
#         self.children.append(node)
        
# class Tree():
#     def __init__(self, root):

class Node():
    def __init__(self, splitting_attr_ind=None, splitting_attr_value=None, label=None):
        self.splitting_attr_ind = splitting_attr_ind
        self.splitting_attr_value = splitting_attr_value
        self.label = label
        
    def set_label(self, label):
        self.label = label
    
    def set_splitting_attr(self, ind, value):
        self.splitting_attr_ind = ind
        self.splitting_attr_value = value
        
class Tree():
    def __init__(self):
        
        self.node = Node()
        self.left = None
        self.right = None
        
    def append_left(self, node):
        self.left = node
        
    def append_right(self, node):
        self.right = node

In [98]:
def id3(examples, target_attr, attributes):
    root = Tree()
    
    if are_positive(target_attr):
        root.node.set_label(1)
        return root
    
    if are_negative(target_attr):
        root.node.set_label(0)
        return root
    
    if not attributes:
        root.node.set_label(most_common_value(target_attr))
        return root
    else:
        gains = get_gains(examples, target_attr)
        A = max(gains)
        
        root.node.set_splitting_attr(ind=gain.index(max(gain)), value=A)
        
        vals = get_all_values()
        mean = eval_mean(vals)
        
        new_examples = []
        
        for each in set(vals):
            
            if each < mean:
                root.node.append_left()
                
                new_examples = subset_satisfied(examples, mean, 'l')
                
            else:
                root.node.append_right()
                
                new_examples = subset_satisfied(examples, mean, 'g')
                
            if not new_examples:
                
                root.node.add_leaf(label=most_common_value(target_attr))
            else:
                
                root.append_left(id3(new_examples, target_attr, attributes))

    return root
    

In [101]:
id3(x_train, y_train, [i for i in range(32)])

Ai  [[10.8, 9.71, 68.77, 357.6, 0.09594, 0.05736, 0.02531, 0.01698, 0.1381, 0.064, 0.1728, 0.4064, 1.126, 11.48, 0.007809, 0.009816, 0.01099, 0.005344, 0.01254, 0.00212, 11.6, 12.02, 73.66, 414.0, 0.1436, 0.1257, 0.1047, 0.04603, 0.209, 0.07699]]
pos samples []
temp  0.0


AssertionError: 

In [84]:
len(get_positive_samples(x_train, y_train))

225